<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/ML-AI-for-the-Working-Analyst/Semana10/Semana10_1_Working_Analyst_NN_Img_Numerico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANTE: Antes de iniciar cambiar entorno de ejecución a GPU

# Introducción

Los datos que utilizaremos combinan 2 tipos de datos que hemos usado previamente pero no en conjunto: Imágenes y Características.

El dataset se compone de datos de propiedades en Francia. En los datos tenemos variables predictoras como el tamaño del inmueble, el tamaño de terreno que incluye, número de baños, etc. Además, cada inmueble cuenta con imágenes, que serán vectorizadas para tratar de obtener una buena predicción del precio de los mismos.

Datos en el siguiente [enlace](https://challengedata.ens.fr/participants/challenges/68/).

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana10/X_train_J01Z4CN.csv', index_col='id_annonce')

In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana10/y_train_OXxrJt1.csv', index_col='id_annonce')

In [ ]:
features

In [ ]:
targets

In [ ]:
# Ver en que tipo de dato se guardan los índices

targets.index.dtype

In [ ]:
# Revisar cantidad de datos nulos (bruto o porcentual)

to_drop = features.isnull().sum()/len(features)
to_drop

In [ ]:
to_drop[to_drop > .40]

In [ ]:
index_to_drop = to_drop[to_drop > .40].index.drop('land_size')

In [ ]:
index_to_drop

In [ ]:
features.drop(columns=index_to_drop, inplace=True)
features

In [ ]:
# Imprimir la cantidad de Columnas que nos quedan
features.columns

¿Que datos sugieres tomar y porqué?

In [ ]:
# Limpieza de valores nulos por filas
features = features.dropna(axis=0)


In [ ]:
features.property_type.unique()

In [ ]:
features.property_type.value_counts()

In [ ]:
# Tipo de propiedades que casi no aparecen
to_drop_2 = features.property_type.value_counts()[4:].index
to_drop_2

In [ ]:
features = features[~features.property_type.isin(to_drop_2)]
features

¿Vale la pena tomar datos geográficos? ¿Bajo que condiciones?

In [ ]:
# Drop de las columnas 'approximate_latitude', 'approximate_longitude', 'city', 'postal_code'

features = features.drop(columns=['approximate_latitude', 'approximate_longitude', 'city', 'postal_code'])

In [ ]:
features.property_type.value_counts()

In [ ]:
# Dummies

features = pd.get_dummies(features)

In [ ]:
features

In [ ]:
features.columns

In [ ]:
# Eliminamos bastante, pero nos quedan 10,000 datos para trabajar con imágenes, un número decente
features.isnull().sum().sum()

# Carga de imágenes

Aquí utilizaremos los ID de los features, para extraer las imágenes que nos convengan del archivo zip, por ahora solo extraremos una imagen por cada índice

In [ ]:
import zipfile
import os

from skimage import io
from skimage.transform import resize

from matplotlib import image
import matplotlib.pyplot as plt

In [ ]:
archive = zipfile.ZipFile('/content/drive/MyDrive/Curso-WorkingAnalyst/semana10/reduced_images_ILB.zip', 'r')

In [ ]:
# Extrae todo lo que hay en el .zip. Es temporal, a menos que a ".extractall" le pasen como argumento
# alguna dirección donde quieren guardar sus archivos en Drive.
# Ejemplo: archive.extractall('/content/drive/su_carpeta')

# 1 min 7 seg
archive.extractall()

In [ ]:
# Lista de los nombres de carpetas, para relacionarlo con los ID de los csv cargados al principio
carpetas_imgs = features.index

In [ ]:
# Cargamos las imagenes (46s) TENER CUIDADO DE USARLO CON NO TANTAS IMAGENES

indexs = []
list_imgs = []
for image_id in carpetas_imgs:
  path = '/content/reduced_images/train/' + 'ann_' + str(image_id)
  
  # Obtiene todas las imagenes dentro del path dado
  img_names = os.listdir(path)

  indexs.append(image_id)

  image_array = image.imread(path + '/' + img_names[0])
  #image_array = resize(image_array, (160, 120)) # En general esta linea puede dar problemas de RAM, dependiendo hay que decidir como reescalar


  list_imgs.append([image_array])

In [ ]:
print(len(list_imgs))
print('-------')
print(len(indexs))

In [ ]:
df_images = pd.DataFrame(list_imgs, index=indexs, columns=['image_array'])

In [ ]:
df_images

In [ ]:
df_images.iloc[0,0].shape

In [ ]:
# 58 seg
df_images['image_array'] = df_images['image_array'].apply(lambda x: resize(x, (96,128)))

In [ ]:
df_images.iloc[0,0].shape

In [ ]:
plt.imshow(df_images.iloc[0,0])

¿Porqué no concatenar los DataFrames?

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_id, test_id= train_test_split(indexs, train_size=0.75, random_state=0)

In [ ]:
len(train_id)

In [ ]:
len(test_id)

In [ ]:
train_imgs = df_images.loc[train_id, :]
test_imgs = df_images.loc[test_id, :]

In [ ]:
test_imgs

In [ ]:
train_targets = targets.loc[train_id, :]
test_targets = targets.loc[test_id, :]

In [ ]:
test_targets

A veces hay problemas para utilizar estos datos para alimentar modelos, convirtamos los datos a array

In [ ]:
train_imgs_to_use = np.array([array for array in train_imgs['image_array']])

In [ ]:
test_imgs_to_use = np.array([array for array in test_imgs['image_array']])

In [ ]:
train_imgs_to_use.shape

# Creación de una red neuronal que usaremos

In [ ]:
model = keras.Sequential([
                             
    keras.layers.BatchNormalization(),
    #filter = the dimensionality of the output space
    #kernel_size = specifying the height and width of the 2D convolution window
    keras.layers.Conv2D(filters=16, input_shape = (96, 128, 3), kernel_size=(3,3), activation="relu", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),  #Downsamples the input along its spatial dimensions by taking the maximum value over a window (size defined by pool_size) for each channel.

    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu', name='dense_1', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(84, activation='relu', name='dense_2', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.10),
    keras.layers.Dense(1, activation='linear', name='dense_3')

])

In [ ]:
model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))  #'rmsprop'

In [ ]:
model.fit(train_imgs_to_use, np.array(train_targets), epochs=200)

In [ ]:
model.save('/content/drive/MyDrive/Curso-WorkingAnalyst/semana10')

En lugar de arreglos, también podríamos haber utilizado "tensores" para alimentar nuestro modelo. A veces pueden ser mas útiles unos que otros.

In [ ]:
tensor_train = tf.convert_to_tensor(train_imgs_to_use)

In [ ]:
tensor_test = tf.convert_to_tensor(test_imgs_to_use)

In [ ]:
type(tensor_train)

In [ ]:
tensor_train.shape

# Cargado de los pesos de un modelo ya entrenado

In [ ]:
model.load_weights('/content/drive/MyDrive/Curso-WorkingAnalyst/semana10')

In [ ]:
np.array(train_targets[0:20])

In [ ]:
model.predict(test_imgs_to_use)[:20]

In [ ]:
plt.plot(range(20), np.array(test_targets)[:20], c='r')
plt.plot(range(20), model.predict(test_imgs_to_use)[:20])

In [ ]:
train_pred = model.predict(tensor_train)

In [ ]:
train_targets['price']

In [ ]:
train_pred

In [ ]:
plt.plot(range(100),train_targets['price'][1100:1200]) #range(len(train_targets))
plt.plot(range(100),train_pred[1100:1200])

In [ ]:
keras.utils.plot_model( 
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TD",
    dpi=50,
)